In [1]:
from ultralytics import YOLO
import pandas as pd

import torch

from FedYOLO.config import HOME, SPLITS_CONFIG, SERVER_CONFIG

DATASET_NAME = SPLITS_CONFIG['dataset_name']
NUM_ROUNDS = SERVER_CONFIG['rounds']
NUM_ROUNDS = 1

server_model_weights_path = f"{HOME}/weights/model_weights___round_{NUM_ROUNDS}_{DATASET_NAME}_WOHO.pt"

In [2]:
from flwr.common import parameters_to_ndarrays
from flwr.common import ndarrays_to_parameters

dummy_server_model_weights_path = f"{HOME}/weights/model_weights___round_{NUM_ROUNDS}_{DATASET_NAME}_dummy.pt"

def get_parameters(net):
    return [val.cpu().numpy() for _, val in net.model.state_dict().items()]

def set_parameters(net, parameters):
    # Convert parameters back to numpy arrays
    ndarrays = parameters_to_ndarrays(parameters)
    
    # Get the current state dict
    state_dict = net.model.state_dict()
    
    # Update the state dict with the new numpy arrays
    for (name, param), ndarray in zip(state_dict.items(), ndarrays):
        # Convert numpy array back to torch tensor
        state_dict[name] = torch.tensor(ndarray)
    
    # Load the updated state dict
    net.model.load_state_dict(state_dict)

    # save the model
    state_dict = net.model.state_dict()

    # save the model
    torch.save(state_dict, dummy_server_model_weights_path)


    return state_dict

model = YOLO()
ndarrays = get_parameters(model)
parameters = ndarrays_to_parameters(ndarrays)

# Reverse the process from parameters to ndarrays to model
state_dict = set_parameters(model, parameters)
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [3]:
original_weights = torch.load(server_model_weights_path)
dummy_weights = torch.load(dummy_server_model_weights_path)

model = YOLO()
model.model.load_state_dict(dummy_weights)
# model.model.load_state_dict(original_weights)

def load_selective_weights(model, weights):
    skipped_layers = 0

    for name, param in model.model.state_dict().items():
        if name in weights and model.model.state_dict()[name].size() == weights[name].size():
            model.model.state_dict()[name].copy_(weights[name])
        else:
            skipped_layers += 1

    print(f"Total layers skipped: {skipped_layers}")

    state_dict = model.model.state_dict()
    torch.save(state_dict, dummy_server_model_weights_path)

    return model

In [4]:
# import torch

# # Load weights
# original_weights = torch.load(server_model_weights_path)
# dummy_weights = torch.load(dummy_server_model_weights_path)

# # Compare shapes individually
# for key in original_weights.keys():
#     if key in dummy_weights:
#         original_shape = original_weights[key].shape
#         dummy_shape = dummy_weights[key].shape
#         if original_shape != dummy_shape:
#             print('###################')
#             print(f"Layer: {key}, Original Shape: {original_shape}, Dummy Shape: {dummy_shape}")
#             print('###################')
#         else:
#             print(f"Layer: {key}, Original Shape: {original_shape}, Dummy Shape: {dummy_shape}")


# print()
# print()

# # Comparing the overall number of layers
# original_layers = len(original_weights.keys())
# dummy_layers = len(dummy_weights.keys())
# print(f"Original Layers: {original_layers}, Dummy Layers: {dummy_layers}")

In [5]:
# server_model = YOLO(server_model_weights_path)
# server_model_client0_metrics = server_model.val(data=f'{HOME}/datasets/{DATASET_NAME}/partitions/client_0/data.yaml', verbose=True)
# server_model_client1_metrics = server_model.val(data=f'{HOME}/datasets/{DATASET_NAME}/partitions/client_1/data.yaml', verbose=True)
# server_model_global_metrics = server_model.val(data=f'{HOME}/datasets/{DATASET_NAME}/data.yaml', verbose=True)

# server_model_client0_table = pd.DataFrame({
#     'Class': list(server_model_client0_metrics.names.values()),
#     'mAP@0.5:0.95': server_model_client0_metrics.box.maps.tolist()
# })

# server_model_client1_table = pd.DataFrame({
#     'Class': list(server_model_client1_metrics.names.values()),
#     'mAP@0.5:0.95': server_model_client1_metrics.box.maps.tolist()
# })

# server_model_global_table = pd.DataFrame({
#     'Class': list(server_model_global_metrics.names.values()),
#     'mAP@0.5:0.95': server_model_global_metrics.box.maps.tolist()
# })

# server_model_combined_table = pd.merge(server_model_client0_table, server_model_client1_table, on='Class', how='inner')
# print(server_model_combined_table)

In [ ]:
from ultralytics import YOLO
from extract_final_save_from_client import extract_results_path

import pandas as pd

from FedYOLO.config import HOME, SPLITS_CONFIG, SERVER_CONFIG

DATASET_NAME = SPLITS_CONFIG['dataset_name']
NUM_ROUNDS = SERVER_CONFIG['rounds']

# #! HUGE DIFFERENCES BETWEEN SYSTEMS REGARDING FILES PATHS AND LOGGING. NEED TO IN-DEPTH TEST THIS.

# #####################
# # CLIENT EVALUATION #
# #####################

# def get_client_metrics(client_number, dataset_name, home_path):
#     # Extract paths
#     client_model_weights_path = extract_results_path(f"{home_path}/logs/client_{client_number}_log_{dataset_name}.txt")
#     weights = f"{home_path}/{client_model_weights_path}/weights/best.pt"
    
#     # Load and validate local model
#     client_model = YOLO()
#     state_dict = torch.load(weights)['model'].state_dict()
#     # new_state_dict = {k.replace('model.', 'model.model.'): v for k, v in state_dict.items()}
#     # client_model.load_state_dict(torch.load(weights)['model'].state_dict())
#     client_model.model.load_state_dict(state_dict)
#     # client_model.model.load_state_dict(torch.load(weights), strict=False)
#     client_metrics = client_model.val(data=f'{home_path}/datasets/{dataset_name}/partitions/client_{client_number}/data.yaml', verbose=True)
    
#     # Create local model metrics table
#     client_table = pd.DataFrame({
#         'Class': list(client_metrics.names.values()),
#         'mAP@0.5:0.95': client_metrics.box.maps.tolist()
#     })
    
#     # Extract global model weights
#     client_global_model_weights_path = extract_results_path(f"{home_path}/logs/client_{client_number}_log_{dataset_name}.txt")
#     global_weights = f"{home_path}/{client_global_model_weights_path}/weights/best.pt"
    
#     # Load and validate global model
#     client_global_model = YOLO(global_weights)
#     client_global_metrics = client_global_model.val(data=f'{home_path}/datasets/{dataset_name}/data.yaml', verbose=False)
    
#     # Create global model metrics table
#     client_global_table = pd.DataFrame({
#         'Class': list(client_global_metrics.names.values()),
#         'mAP@0.5:0.95': client_global_metrics.box.maps.tolist()
#     })
    
#     # Combine local and global model metrics
#     combined_table = pd.merge(client_table, client_global_table, on='Class', how='inner')
#     combined_table.columns = ['Class', 'mAP@0.5:0.95_local', 'mAP@0.5:0.95_global']

#     del client_model
#     del client_global_model
    
#     return combined_table

# client_0_metrics_table = get_client_metrics(0, DATASET_NAME, HOME)
# client_1_metrics_table = get_client_metrics(1, DATASET_NAME, HOME)
# combined_table = pd.merge(client_0_metrics_table, client_1_metrics_table, on='Class', how='inner')
# combined_table.columns = ['Class', 'mAP@0.5:0.95_local_0', 'mAP@0.5:0.95_global_0', 'mAP@0.5:0.95_local_1', 'mAP@0.5:0.95_global_1']
# print()
# print()
# print('##############################')
# print('# FINAL CONSOLIDATED METRICS #')
# print('##############################')
# print(combined_table.to_string(index=False))

RuntimeError: Error(s) in loading state_dict for DetectionModel:
	size mismatch for model.23.cv3.0.0.1.conv.weight: copying a param with shape torch.Size([64, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([80, 64, 1, 1]).
	size mismatch for model.23.cv3.0.0.1.bn.weight: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.23.cv3.0.0.1.bn.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.23.cv3.0.0.1.bn.running_mean: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.23.cv3.0.0.1.bn.running_var: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.23.cv3.0.1.0.conv.weight: copying a param with shape torch.Size([64, 1, 3, 3]) from check

In [17]:
client_model_weights_path = extract_results_path(f"{HOME}/logs/client_1_log_pills.txt")
print(client_model_weights_path)

runs/detect/train70


In [18]:
weights = f"{HOME}/runs/detect/train70/weights/best.pt"
model = YOLO(weights)

In [25]:
model = YOLO('yolo11n.yaml')
model.nc = 8 
model.model.load_state_dict(torch.load(weights)['model'].state_dict(), strict=False)

RuntimeError: Error(s) in loading state_dict for DetectionModel:
	size mismatch for model.23.cv3.0.0.1.conv.weight: copying a param with shape torch.Size([64, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([80, 64, 1, 1]).
	size mismatch for model.23.cv3.0.0.1.bn.weight: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.23.cv3.0.0.1.bn.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.23.cv3.0.0.1.bn.running_mean: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.23.cv3.0.0.1.bn.running_var: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.23.cv3.0.1.0.conv.weight: copying a param with shape torch.Size([64, 1, 3, 3]) from check

In [26]:
# Get the current state dict of your model
model_state_dict = model.model.state_dict()

# Create a new state dict with only the matching keys
new_state_dict = {k: v for k, v in state_dict.items() if k in model_state_dict and v.shape == model_state_dict[k].shape}

# Load the new state dict
model.model.load_state_dict(new_state_dict, strict=False)

_IncompatibleKeys(missing_keys=['model.0.conv.weight', 'model.0.bn.weight', 'model.0.bn.bias', 'model.0.bn.running_mean', 'model.0.bn.running_var', 'model.1.conv.weight', 'model.1.bn.weight', 'model.1.bn.bias', 'model.1.bn.running_mean', 'model.1.bn.running_var', 'model.2.cv1.conv.weight', 'model.2.cv1.bn.weight', 'model.2.cv1.bn.bias', 'model.2.cv1.bn.running_mean', 'model.2.cv1.bn.running_var', 'model.2.cv2.conv.weight', 'model.2.cv2.bn.weight', 'model.2.cv2.bn.bias', 'model.2.cv2.bn.running_mean', 'model.2.cv2.bn.running_var', 'model.2.m.0.cv1.conv.weight', 'model.2.m.0.cv1.bn.weight', 'model.2.m.0.cv1.bn.bias', 'model.2.m.0.cv1.bn.running_mean', 'model.2.m.0.cv1.bn.running_var', 'model.2.m.0.cv2.conv.weight', 'model.2.m.0.cv2.bn.weight', 'model.2.m.0.cv2.bn.bias', 'model.2.m.0.cv2.bn.running_mean', 'model.2.m.0.cv2.bn.running_var', 'model.3.conv.weight', 'model.3.bn.weight', 'model.3.bn.bias', 'model.3.bn.running_mean', 'model.3.bn.running_var', 'model.4.cv1.conv.weight', 'model.4.